In [41]:
from langchain_community.document_loaders import DirectoryLoader , PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_classic.embeddings import HuggingFaceBgeEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import RetrievalQA


In [11]:
# load the documents 

loader = DirectoryLoader(r"E:\RAG\RAG\us-census",glob = "*.pdf",
                         loader_cls=PyPDFLoader)
docs = loader.load()
docs[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-09-09T07:52:17-04:00', 'author': 'U.S. Census Bureau', 'keywords': 'acsbr-015', 'moddate': '2023-09-12T14:44:47+01:00', 'title': 'Health Insurance Coverage Status and Type by Geography: 2021 and 2022', 'trapped': '/false', 'source': 'E:\\RAG\\RAG\\us-census\\acsbr-015.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Heal

In [73]:
#Splitt the document into chunks 

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 800,chunk_overlap =200)
text = text_splitter.split_documents(docs)
print(text[120].page_content)

U.S. Census Bureau  3
Utah had poverty rates in the 
lowest poverty map category, 
less than 10.0 percent in 2022.
9 
Alabama, Arkansas, Kentucky, 
Louisiana, Mississippi, New Mexico, 
Oklahoma, West Virginia, and 
Puerto Rico had 2022 poverty 
rates of 15.0 percent or higher 
(Figure 2).
In 2022, no states had a significant 
increase in poverty, whereas nine 
9 The 2022 poverty rates for Washington 
(10.0 percent), Hawaii (10.2 percent), 
Vermont (10.4 percent), and Rhode Island 
(10.8 percent) were not statistically different 
from 10.0 percent.
states and the District of Columbia 
showed decreases in poverty rates 
compared to 2021 (Figure 3). 
Forty-one states did not signifi-
cantly change.
While the overall national 2022 
poverty rate was 12.6 percent, 
there was variability among


In [17]:
# convert the text into vectors using hugging face embedding

hf = HuggingFaceBgeEmbeddings(
    model_name = "BAAI/bge-small-en",
    model_kwargs = {"device": "cpu"},
encode_kwargs = {"normalize_embeddings": True}
)

# storing into vector DB FAISS
vectorstore=FAISS.from_documents(text[:120],hf)


In [18]:
query = "what is health insurance coverage"
relevant_documents = vectorstore.similarity_search(query)
print(relevant_documents[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines


In [52]:
retriever = vectorstore.as_retriever(search_type = "similarity",
                                     search_kwargs = {"k":3})

In [53]:
import os 
from dotenv import load_dotenv
load_dotenv()

load_dotenv()
groq_api_key = os.environ["GROQ_API_KEY"]


In [55]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.3-70b-versatile"
)



In [56]:
prompt_template = """
use the following piece of context to answer the asked question.
if the relevant answer is not present say - the information is not provided

{context}
Question:{question}

helpful answer:
 """

In [46]:
prompt = ChatPromptTemplate.from_template(prompt_template)

In [57]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [74]:
query="""which place had the lowest 
2022 rate at 7.2 percent"""

In [75]:
result = retrievalQA.invoke({"query": query})
print(result['result'])

New Hampshire had the lowest 2022 rate at 7.2 percent.
